In [1]:
# Cell 1: Imports and setup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Optionally set display options for wide dataframes
pd.set_option('display.max_columns', None)

In [2]:
# Cell 2: Load data from CSV files
tourney_results = pd.read_csv('../raw_data/MNCAATourneyDetailedResults.csv')
tourney_seeds = pd.read_csv('../raw_data/MNCAATourneySeeds.csv')
kenpom = pd.read_csv('../pre_tourney_data/KenPom-Rankings-Updated.csv')
enhanced_stats = pd.read_csv('../pre_tourney_data/EnhancedTournamentStats.csv')
teams = pd.read_csv('../raw_data/MTeams.csv')

print("Tournament Results shape:", tourney_results.shape)
print("Tournament Seeds shape:", tourney_seeds.shape)
print("KenPom Rankings shape:", kenpom.shape)
print("Enhanced Stats shape:", enhanced_stats.shape)
print("Teams shape:", teams.shape)


Tournament Results shape: (1382, 34)
Tournament Seeds shape: (2558, 3)
KenPom Rankings shape: (7624, 5)
Enhanced Stats shape: (7981, 36)
Teams shape: (380, 4)


In [3]:
# Cell 3: Inspect columns and sample data

print("Tournament Results columns:", tourney_results.columns.tolist())
print("Tournament Seeds columns:", tourney_seeds.columns.tolist())
print("KenPom Rankings columns:", kenpom.columns.tolist())
print("Enhanced Stats columns:", enhanced_stats.columns.tolist())
print("Teams columns:", teams.columns.tolist())

# Show samples to verify team identifier columns
print("\nTournament Results sample (WTeamID and LTeamID):")
print(tourney_results[['WTeamID', 'LTeamID']].head())

print("\nTournament Seeds sample (Seed and TeamID):")
print(tourney_seeds[['Seed', 'TeamID']].head())

print("\nKenPom Rankings sample (TeamID and OrdinalRank):")
print(kenpom[['TeamID', 'OrdinalRank']].head())

print("\nEnhanced Stats sample (TeamID and TeamName):")
print(enhanced_stats[['TeamID', 'TeamName']].head())

print("\nTeams sample:")
print(teams.head())


Tournament Results columns: ['Season', 'DayNum', 'WTeamID', 'WScore', 'LTeamID', 'LScore', 'WLoc', 'NumOT', 'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR', 'WAst', 'WTO', 'WStl', 'WBlk', 'WPF', 'LFGM', 'LFGA', 'LFGM3', 'LFGA3', 'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF']
Tournament Seeds columns: ['Season', 'Seed', 'TeamID']
KenPom Rankings columns: ['Season', 'RankingDayNum', 'SystemName', 'TeamID', 'OrdinalRank']
Enhanced Stats columns: ['Season', 'TeamID', 'TeamName', 'Score', 'Poss', 'ORtg', 'DRtg', 'AdjO', 'AdjD', 'WinGame', 'HomeGame', 'AwayGame', 'NeutralGame', 'ScoreMargin', 'AdjNetRtg', 'SOS_NetRtg', 'SOS_ORtg', 'SOS_DRtg', 'Expected Win%', 'ClutchWin%', 'ThreePtRate', 'FTRate', 'AstRate', 'TORate', 'ORRate', 'DRRate', 'ScoreStdDev', 'MarginStdDev', 'ORtgStdDev', 'DRtgStdDev', 'HomeWin%', 'AwayWin%', 'NeutralWin%', 'HomeAwayORtgDiff', 'Last10Win%', 'Last10Games']
Teams columns: ['TeamID', 'TeamName', 'FirstD1Season', 'LastD1Season']

Tou

In [4]:
# Cell 4: Verify team identifier consistency

# Get unique team IDs from tournament results (both winners and losers)
result_team_ids = set(tourney_results['WTeamID']).union(set(tourney_results['LTeamID']))
print("Unique team IDs in Tournament Results (sample):", sorted(list(result_team_ids))[:10], " ... total", len(result_team_ids))

# Unique team IDs from Tournament Seeds
seeds_team_ids = set(tourney_seeds['TeamID'])
print("Unique team IDs in Tournament Seeds (sample):", sorted(list(seeds_team_ids))[:10], " ... total", len(seeds_team_ids))

# Unique team IDs from KenPom Rankings
kenpom_team_ids = set(kenpom['TeamID'])
print("Unique team IDs in KenPom Rankings (sample):", sorted(list(kenpom_team_ids))[:10], " ... total", len(kenpom_team_ids))

# Unique team IDs from Enhanced Stats
enhanced_team_ids = set(enhanced_stats['TeamID'])
print("Unique team IDs in Enhanced Stats (sample):", sorted(list(enhanced_team_ids))[:10], " ... total", len(enhanced_team_ids))

# Unique team IDs from Teams file
teams_team_ids = set(teams['TeamID'])
print("Unique team IDs in Teams file (sample):", sorted(list(teams_team_ids))[:10], " ... total", len(teams_team_ids))

# Check intersection between Tournament Results and Teams file
common_ids = result_team_ids.intersection(teams_team_ids)
print("\nCommon team IDs between Tournament Results and Teams:", sorted(list(common_ids))[:10], " ... total", len(common_ids))


Unique team IDs in Tournament Results (sample): [1101, 1102, 1103, 1104, 1105, 1106, 1107, 1110, 1111, 1112]  ... total 268
Unique team IDs in Tournament Seeds (sample): [1101, 1102, 1103, 1104, 1105, 1106, 1107, 1108, 1110, 1111]  ... total 309
Unique team IDs in KenPom Rankings (sample): [1101, 1102, 1103, 1104, 1105, 1106, 1107, 1108, 1110, 1111]  ... total 371
Unique team IDs in Enhanced Stats (sample): [1101, 1102, 1103, 1104, 1105, 1106, 1107, 1108, 1110, 1111]  ... total 371
Unique team IDs in Teams file (sample): [1101, 1102, 1103, 1104, 1105, 1106, 1107, 1108, 1109, 1110]  ... total 380

Common team IDs between Tournament Results and Teams: [1101, 1102, 1103, 1104, 1105, 1106, 1107, 1110, 1111, 1112]  ... total 268


In [5]:
# Cell 5: Check season ranges in each dataset
print("Tournament Results seasons:", sorted(tourney_results['Season'].unique()))
print("Tournament Seeds seasons:", sorted(tourney_seeds['Season'].unique()))
print("KenPom Rankings seasons:", sorted(kenpom['Season'].unique()))
print("Enhanced Stats seasons:", sorted(enhanced_stats['Season'].unique()))
# Note: Teams file doesn't have season information.


Tournament Results seasons: [np.int64(2003), np.int64(2004), np.int64(2005), np.int64(2006), np.int64(2007), np.int64(2008), np.int64(2009), np.int64(2010), np.int64(2011), np.int64(2012), np.int64(2013), np.int64(2014), np.int64(2015), np.int64(2016), np.int64(2017), np.int64(2018), np.int64(2019), np.int64(2021), np.int64(2022), np.int64(2023), np.int64(2024)]
Tournament Seeds seasons: [np.int64(1985), np.int64(1986), np.int64(1987), np.int64(1988), np.int64(1989), np.int64(1990), np.int64(1991), np.int64(1992), np.int64(1993), np.int64(1994), np.int64(1995), np.int64(1996), np.int64(1997), np.int64(1998), np.int64(1999), np.int64(2000), np.int64(2001), np.int64(2002), np.int64(2003), np.int64(2004), np.int64(2005), np.int64(2006), np.int64(2007), np.int64(2008), np.int64(2009), np.int64(2010), np.int64(2011), np.int64(2012), np.int64(2013), np.int64(2014), np.int64(2015), np.int64(2016), np.int64(2017), np.int64(2018), np.int64(2019), np.int64(2021), np.int64(2022), np.int64(2023), 

In [6]:
# Cell 6: Merge check for seed lookup on a sample game
sample_game = tourney_results.iloc[0]
season = sample_game['Season']
w_team = sample_game['WTeamID']
l_team = sample_game['LTeamID']

print("Sample game details:")
print(sample_game[['Season', 'WTeamID', 'LTeamID', 'WScore', 'LScore']])

# Lookup seeds for the winning and losing teams
w_seed = tourney_seeds[(tourney_seeds['Season'] == season) & (tourney_seeds['TeamID'] == w_team)]
l_seed = tourney_seeds[(tourney_seeds['Season'] == season) & (tourney_seeds['TeamID'] == l_team)]

print("\nWinning team seed lookup:")
print(w_seed)

print("\nLosing team seed lookup:")
print(l_seed)


Sample game details:
Season     2003
WTeamID    1421
LTeamID    1411
WScore       92
LScore       84
Name: 0, dtype: object

Winning team seed lookup:
      Season  Seed  TeamID
1186    2003  X16b    1421

Losing team seed lookup:
      Season  Seed  TeamID
1185    2003  X16a    1411


In [7]:
# Cell 7: Feature Engineering – Create game dataset
def create_game_dataset(results, seeds, kenpom_data, enhanced_data):
    games = []
    
    for _, game in results.iterrows():
        season = game['Season']
        day = game['DayNum']
        w_team = game['WTeamID']
        l_team = game['LTeamID']
        
        # Lookup seed information and convert seed string to numeric value
        try:
            w_seed_str = seeds[(seeds['Season'] == season) & (seeds['TeamID'] == w_team)]['Seed'].values[0]
            l_seed_str = seeds[(seeds['Season'] == season) & (seeds['TeamID'] == l_team)]['Seed'].values[0]
            w_seed = int(w_seed_str[1:3])
            l_seed = int(l_seed_str[1:3])
        except Exception as e:
            continue  # Skip game if seed info is missing
        
        # Lookup KenPom rankings
        try:
            w_kenpom = kenpom_data[(kenpom_data['Season'] == season) & (kenpom_data['TeamID'] == w_team)]['OrdinalRank'].values[0]
            l_kenpom = kenpom_data[(kenpom_data['Season'] == season) & (kenpom_data['TeamID'] == l_team)]['OrdinalRank'].values[0]
        except Exception as e:
            w_kenpom, l_kenpom = 400, 400  # Default high rank if missing
        
        # Lookup Enhanced Stats
        try:
            w_stats = enhanced_data[(enhanced_data['Season'] == season) & (enhanced_data['TeamID'] == w_team)].iloc[0]
            l_stats = enhanced_data[(enhanced_data['Season'] == season) & (enhanced_data['TeamID'] == l_team)].iloc[0]
        except Exception as e:
            continue  # Skip game if enhanced stats are missing
        
        game_dict = {
            'Season': season,
            'DayNum': day,
            'WTeamID': w_team,
            'LTeamID': l_team,
            'WScore': game['WScore'],
            'LScore': game['LScore'],
            'ScoreDiff': game['WScore'] - game['LScore'],
            'WSeed': w_seed,
            'LSeed': l_seed,
            'SeedDiff': l_seed - w_seed,  # A higher numeric seed means a lower seed ranking
            'WKenPom': w_kenpom,
            'LKenPom': l_kenpom,
            'KenPomDiff': l_kenpom - w_kenpom,
            'Upset': 1 if w_seed > l_seed else 0
        }
        
        # Enhanced stats features to add
        enhanced_features = ['AdjO', 'AdjD', 'AdjNetRtg', 'SOS_NetRtg', 'Expected Win%', 
                             'ThreePtRate', 'FTRate', 'AstRate', 'TORate', 'ORRate', 'DRRate',
                             'ScoreStdDev', 'MarginStdDev', 'ORtgStdDev', 'DRtgStdDev',
                             'HomeWin%', 'AwayWin%', 'NeutralWin%', 'Last10Win%']
        
        for feature in enhanced_features:
            if feature in w_stats and feature in l_stats:
                game_dict[f'W_{feature}'] = w_stats[feature]
                game_dict[f'L_{feature}'] = l_stats[feature]
                game_dict[f'Diff_{feature}'] = w_stats[feature] - l_stats[feature]
        
        # For metrics where a lower value is better, flip the difference sign
        lower_is_better = ['KenPomDiff', 'Diff_AdjD', 'Diff_TORate']
        for feature in lower_is_better:
            if feature in game_dict:
                game_dict[feature] = -game_dict[feature]
        
        games.append(game_dict)
        
    return pd.DataFrame(games)

# Create the game dataset
game_df = create_game_dataset(tourney_results, tourney_seeds, kenpom, enhanced_stats)
print("Game dataset shape:", game_df.shape)
game_df.head()


Game dataset shape: (1382, 71)


,Season,DayNum,WTeamID,LTeamID,WScore,LScore,ScoreDiff,WSeed,LSeed,SeedDiff,WKenPom,LKenPom,KenPomDiff,Upset,W_AdjO,L_AdjO,Diff_AdjO,W_AdjD,L_AdjD,Diff_AdjD,W_AdjNetRtg,L_AdjNetRtg,Diff_AdjNetRtg,W_SOS_NetRtg,L_SOS_NetRtg,Diff_SOS_NetRtg,W_Expected Win%,L_Expected Win%,Diff_Expected Win%,W_ThreePtRate,L_ThreePtRate,Diff_ThreePtRate,W_FTRate,L_FTRate,Diff_FTRate,W_AstRate,L_AstRate,Diff_AstRate,W_TORate,L_TORate,Diff_TORate,W_ORRate,L_ORRate,Diff_ORRate,W_DRRate,L_DRRate,Diff_DRRate,W_ScoreStdDev,L_ScoreStdDev,Diff_ScoreStdDev,W_MarginStdDev,L_MarginStdDev,Diff_MarginStdDev,W_ORtgStdDev,L_ORtgStdDev,Diff_ORtgStdDev,W_DRtgStdDev,L_DRtgStdDev,Diff_DRtgStdDev,W_HomeWin%,L_HomeWin%,Diff_HomeWin%,W_AwayWin%,L_AwayWin%,Diff_AwayWin%,W_NeutralWin%,L_NeutralWin%,Diff_NeutralWin%,W_Last10Win%,L_Last10Win%,Diff_Last10Win%
0,2003,134,1421,1411,92,84,8,16,16,0,273,268,5,0,98.245732,98.005282,0.240450,115.581923,106.099578,-9.482344,-17.336191,-8.094297,-9.241894,-6.586565,-11.486607,4.900043,0.273440,0.576493,-0.303053,0.273123,0.244505,0.028618,0.224213,0.239011,-0.014798,0.534653,0.574124,-0.039471,0.229344,0.215571,-0.013773,0.346304,0.346795,-0.000492,0.653696,0.653205,0.000492,11.638542,12.104146,-0.465604,17.508478,12.203683,5.304795,15.912264,13.052262,2.860003,14.267928,17.075457,-2.807529,0.833333,0.785714,0.047619,0.125000,0.333333,-0.208333,1.00,0.75,0.25,0.5,0.5,0.0
1,2003,136,1112,1436,80,51,29,1,16,15,3,144,-141,0,115.323581,99.568984,15.754597,87.631603,98.680569,11.048965,27.691978,0.888416,26.803562,8.518727,-5.768173,14.286900,0.851071,0.646832,0.204238,0.247695,0.233469,0.014226,0.205784,0.189725,0.016058,0.581861,0.572222,0.009639,0.191534,0.212353,0.020820,0.354462,0.335116,0.019346,0.645538,0.664884,-0.019346,10.379314,11.995176,-1.615862,12.556486,12.601802,-0.045316,12.681628,16.267176,-3.585548,12.721087,12.807808,-0.086720,0.933333,0.900000,0.033333,0.916667,0.500000,0.416667,0.00,0.60,-0.60,0.9,0.8,0.1
2,2003,136,1113,1272,84,71,13,10,7,-3,32,20,12,1,114.247642,109.970116,4.277526,94.476582,92.072583,-2.403999,19.771060,17.897533,1.873527,8.776171,5.593764,3.182406,0.730967,0.753715,-0.022748,0.157966,0.281814,-0.123848,0.231049,0.200833,0.030216,0.571610,0.632546,-0.060936,0.200990,0.195370,-0.005620,0.369991,0.351421,0.018569,0.630009,0.648579,-0.018569,11.968658,8.724435,3.244223,17.188824,10.964565,6.224259,16.114254,8.211086,7.903169,15.979505,13.654340,2.325165,0.800000,0.875000,-0.075000,0.400000,0.636364,-0.236364,0.50,1.00,-0.50,0.6,0.9,-0.3
3,2003,136,1141,1166,79,73,6,11,6,-5,50,27,23,1,108.935115,114.480227,-5.545112,100.850880,92.747474,-8.103406,8.084235,21.732753,-13.648518,-0.064548,0.798965,-0.863513,0.677170,0.870239,-0.193068,0.258149,0.301721,-0.043572,0.242938,0.175143,0.067794,0.586788,0.586061,0.000726,0.252295,0.192410,-0.059885,0.312627,0.319395,-0.006768,0.687373,0.680605,0.006768,13.102006,12.262826,0.839180,13.167451,14.908814,-1.741362,15.485106,13.303453,2.181653,11.459503,14.291319,-2.831816,0.916667,1.000000,-0.083333,0.642857,0.636364,0.006494,1.00,1.00,0.00,0.9,0.8,0.1
4,2003,136,1143,1301,76,74,2,8,9,1,40,48,-8,0,108.429377,111.253386,-2.824009,94.534310,95.967664,1.433354,13.895067,15.285722,-1.390655,6.671880,8.492229,-1.820348,0.658508,0.649670,0.008838,0.258333,0.330110,-0.071777,0.179630,0.217772,-0.038142,0.585120,0.602740,-0.017620,0.199820,0.210303,0.010483,0.315586,0.306401,0.009185,0.684414,0.693599,-0.009185,7.758040,12.696999,-4.938959,10.666692,16.208980,-5.542288,9.924813,14.665568,-4.740755,14.872066,14.890732,-0.018665,0.928571,0.812500,0.116071,0.636364,0.300000,0.336364,0.25,0.50,-0.25,0.6,0.5,0.1


In [8]:
# Cell 8: Create balanced dataset
def create_balanced_dataset(game_df):
    original_df = game_df.copy()
    swapped_df = game_df.copy()
    
    # Swap key columns for the swapped perspective
    cols_to_swap = {
        'WTeamID': 'LTeamID',
        'LTeamID': 'WTeamID',
        'WScore': 'LScore',
        'LScore': 'WScore',
        'WSeed': 'LSeed',
        'LSeed': 'WSeed',
        'WKenPom': 'LKenPom',
        'LKenPom': 'WKenPom'
    }
    
    for col1, col2 in cols_to_swap.items():
        swapped_df[col1], swapped_df[col2] = swapped_df[col2], swapped_df[col1]
    
    # Negate difference columns
    for diff_col in ['ScoreDiff', 'SeedDiff', 'KenPomDiff']:
        if diff_col in swapped_df.columns:
            swapped_df[diff_col] = -swapped_df[diff_col]
    
    # Negate all enhanced stats difference features
    for col in swapped_df.columns:
        if col.startswith('Diff_'):
            swapped_df[col] = -swapped_df[col]
    
    # Set target labels: 1 for original (winner perspective), 0 for swapped
    original_df['Target'] = 1
    swapped_df['Target'] = 0
    
    balanced_df = pd.concat([original_df, swapped_df], ignore_index=True)
    return balanced_df

balanced_df = create_balanced_dataset(game_df)
print("Balanced dataset shape:", balanced_df.shape)
balanced_df.head()


Balanced dataset shape: (2764, 72)


,Season,DayNum,WTeamID,LTeamID,WScore,LScore,ScoreDiff,WSeed,LSeed,SeedDiff,WKenPom,LKenPom,KenPomDiff,Upset,W_AdjO,L_AdjO,Diff_AdjO,W_AdjD,L_AdjD,Diff_AdjD,W_AdjNetRtg,L_AdjNetRtg,Diff_AdjNetRtg,W_SOS_NetRtg,L_SOS_NetRtg,Diff_SOS_NetRtg,W_Expected Win%,L_Expected Win%,Diff_Expected Win%,W_ThreePtRate,L_ThreePtRate,Diff_ThreePtRate,W_FTRate,L_FTRate,Diff_FTRate,W_AstRate,L_AstRate,Diff_AstRate,W_TORate,L_TORate,Diff_TORate,W_ORRate,L_ORRate,Diff_ORRate,W_DRRate,L_DRRate,Diff_DRRate,W_ScoreStdDev,L_ScoreStdDev,Diff_ScoreStdDev,W_MarginStdDev,L_MarginStdDev,Diff_MarginStdDev,W_ORtgStdDev,L_ORtgStdDev,Diff_ORtgStdDev,W_DRtgStdDev,L_DRtgStdDev,Diff_DRtgStdDev,W_HomeWin%,L_HomeWin%,Diff_HomeWin%,W_AwayWin%,L_AwayWin%,Diff_AwayWin%,W_NeutralWin%,L_NeutralWin%,Diff_NeutralWin%,W_Last10Win%,L_Last10Win%,Diff_Last10Win%,Target
0,2003,134,1421,1411,92,84,8,16,16,0,273,268,5,0,98.245732,98.005282,0.240450,115.581923,106.099578,-9.482344,-17.336191,-8.094297,-9.241894,-6.586565,-11.486607,4.900043,0.273440,0.576493,-0.303053,0.273123,0.244505,0.028618,0.224213,0.239011,-0.014798,0.534653,0.574124,-0.039471,0.229344,0.215571,-0.013773,0.346304,0.346795,-0.000492,0.653696,0.653205,0.000492,11.638542,12.104146,-0.465604,17.508478,12.203683,5.304795,15.912264,13.052262,2.860003,14.267928,17.075457,-2.807529,0.833333,0.785714,0.047619,0.125000,0.333333,-0.208333,1.00,0.75,0.25,0.5,0.5,0.0,1
1,2003,136,1112,1436,80,51,29,1,16,15,3,144,-141,0,115.323581,99.568984,15.754597,87.631603,98.680569,11.048965,27.691978,0.888416,26.803562,8.518727,-5.768173,14.286900,0.851071,0.646832,0.204238,0.247695,0.233469,0.014226,0.205784,0.189725,0.016058,0.581861,0.572222,0.009639,0.191534,0.212353,0.020820,0.354462,0.335116,0.019346,0.645538,0.664884,-0.019346,10.379314,11.995176,-1.615862,12.556486,12.601802,-0.045316,12.681628,16.267176,-3.585548,12.721087,12.807808,-0.086720,0.933333,0.900000,0.033333,0.916667,0.500000,0.416667,0.00,0.60,-0.60,0.9,0.8,0.1,1
2,2003,136,1113,1272,84,71,13,10,7,-3,32,20,12,1,114.247642,109.970116,4.277526,94.476582,92.072583,-2.403999,19.771060,17.897533,1.873527,8.776171,5.593764,3.182406,0.730967,0.753715,-0.022748,0.157966,0.281814,-0.123848,0.231049,0.200833,0.030216,0.571610,0.632546,-0.060936,0.200990,0.195370,-0.005620,0.369991,0.351421,0.018569,0.630009,0.648579,-0.018569,11.968658,8.724435,3.244223,17.188824,10.964565,6.224259,16.114254,8.211086,7.903169,15.979505,13.654340,2.325165,0.800000,0.875000,-0.075000,0.400000,0.636364,-0.236364,0.50,1.00,-0.50,0.6,0.9,-0.3,1
3,2003,136,1141,1166,79,73,6,11,6,-5,50,27,23,1,108.935115,114.480227,-5.545112,100.850880,92.747474,-8.103406,8.084235,21.732753,-13.648518,-0.064548,0.798965,-0.863513,0.677170,0.870239,-0.193068,0.258149,0.301721,-0.043572,0.242938,0.175143,0.067794,0.586788,0.586061,0.000726,0.252295,0.192410,-0.059885,0.312627,0.319395,-0.006768,0.687373,0.680605,0.006768,13.102006,12.262826,0.839180,13.167451,14.908814,-1.741362,15.485106,13.303453,2.181653,11.459503,14.291319,-2.831816,0.916667,1.000000,-0.083333,0.642857,0.636364,0.006494,1.00,1.00,0.00,0.9,0.8,0.1,1
4,2003,136,1143,1301,76,74,2,8,9,1,40,48,-8,0,108.429377,111.253386,-2.824009,94.534310,95.967664,1.433354,13.895067,15.285722,-1.390655,6.671880,8.492229,-1.820348,0.658508,0.649670,0.008838,0.258333,0.330110,-0.071777,0.179630,0.217772,-0.038142,0.585120,0.602740,-0.017620,0.199820,0.210303,0.010483,0.315586,0.306401,0.009185,0.684414,0.693599,-0.009185,7.758040,12.696999,-4.938959,10.666692,16.208980,-5.542288,9.924813,14.665568,-4.740755,14.872066,14.890732,-0.018665,0.928571,0.812500,0.116071,0.636364,0.300000,0.336364,0.25,0.50,-0.25,0.6,0.5,0.1,1


In [9]:
# Cell 9: Feature selection
seed_features = ['SeedDiff']
kenpom_features = ['KenPomDiff']
enhanced_diff_features = [col for col in balanced_df.columns if col.startswith('Diff_')]

features = seed_features + kenpom_features + enhanced_diff_features
print("Selected features:", features)
print("Total number of features:", len(features))

# Prepare the feature matrix X and target vector y
X = balanced_df[features]
y = balanced_df['Target']
X.head()


Selected features: ['SeedDiff', 'KenPomDiff', 'Diff_AdjO', 'Diff_AdjD', 'Diff_AdjNetRtg', 'Diff_SOS_NetRtg', 'Diff_Expected Win%', 'Diff_ThreePtRate', 'Diff_FTRate', 'Diff_AstRate', 'Diff_TORate', 'Diff_ORRate', 'Diff_DRRate', 'Diff_ScoreStdDev', 'Diff_MarginStdDev', 'Diff_ORtgStdDev', 'Diff_DRtgStdDev', 'Diff_HomeWin%', 'Diff_AwayWin%', 'Diff_NeutralWin%', 'Diff_Last10Win%']
Total number of features: 21


,SeedDiff,KenPomDiff,Diff_AdjO,Diff_AdjD,Diff_AdjNetRtg,Diff_SOS_NetRtg,Diff_Expected Win%,Diff_ThreePtRate,Diff_FTRate,Diff_AstRate,Diff_TORate,Diff_ORRate,Diff_DRRate,Diff_ScoreStdDev,Diff_MarginStdDev,Diff_ORtgStdDev,Diff_DRtgStdDev,Diff_HomeWin%,Diff_AwayWin%,Diff_NeutralWin%,Diff_Last10Win%
0,0,5,0.240450,-9.482344,-9.241894,4.900043,-0.303053,0.028618,-0.014798,-0.039471,-0.013773,-0.000492,0.000492,-0.465604,5.304795,2.860003,-2.807529,0.047619,-0.208333,0.25,0.0
1,15,-141,15.754597,11.048965,26.803562,14.286900,0.204238,0.014226,0.016058,0.009639,0.020820,0.019346,-0.019346,-1.615862,-0.045316,-3.585548,-0.086720,0.033333,0.416667,-0.60,0.1
2,-3,12,4.277526,-2.403999,1.873527,3.182406,-0.022748,-0.123848,0.030216,-0.060936,-0.005620,0.018569,-0.018569,3.244223,6.224259,7.903169,2.325165,-0.075000,-0.236364,-0.50,-0.3
3,-5,23,-5.545112,-8.103406,-13.648518,-0.863513,-0.193068,-0.043572,0.067794,0.000726,-0.059885,-0.006768,0.006768,0.839180,-1.741362,2.181653,-2.831816,-0.083333,0.006494,0.00,0.1
4,1,-8,-2.824009,1.433354,-1.390655,-1.820348,0.008838,-0.071777,-0.038142,-0.017620,0.010483,0.009185,-0.009185,-4.938959,-5.542288,-4.740755,-0.018665,0.116071,0.336364,-0.25,0.1


In [10]:
# Cell 10: Train-test split by season
seasons = sorted(balanced_df['Season'].unique())
print("All Seasons:", seasons)

# For example: train on all but last 3 seasons, validate on the third-to-last, test on the last 2
train_seasons = seasons[:-3]
val_seasons = [seasons[-3]]
test_seasons = seasons[-2:]

print("Training Seasons:", train_seasons)
print("Validation Season:", val_seasons)
print("Test Seasons:", test_seasons)

X_train = X[balanced_df['Season'].isin(train_seasons)]
y_train = y[balanced_df['Season'].isin(train_seasons)]

X_val = X[balanced_df['Season'].isin(val_seasons)]
y_val = y[balanced_df['Season'].isin(val_seasons)]

X_test = X[balanced_df['Season'].isin(test_seasons)]
y_test = y[balanced_df['Season'].isin(test_seasons)]

print("Training samples:", X_train.shape[0])
print("Validation samples:", X_val.shape[0])
print("Test samples:", X_test.shape[0])


All Seasons: [np.int64(2003), np.int64(2004), np.int64(2005), np.int64(2006), np.int64(2007), np.int64(2008), np.int64(2009), np.int64(2010), np.int64(2011), np.int64(2012), np.int64(2013), np.int64(2014), np.int64(2015), np.int64(2016), np.int64(2017), np.int64(2018), np.int64(2019), np.int64(2021), np.int64(2022), np.int64(2023), np.int64(2024)]
Training Seasons: [np.int64(2003), np.int64(2004), np.int64(2005), np.int64(2006), np.int64(2007), np.int64(2008), np.int64(2009), np.int64(2010), np.int64(2011), np.int64(2012), np.int64(2013), np.int64(2014), np.int64(2015), np.int64(2016), np.int64(2017), np.int64(2018), np.int64(2019), np.int64(2021)]
Validation Season: [np.int64(2022)]
Test Seasons: [np.int64(2023), np.int64(2024)]
Training samples: 2362
Validation samples: 134
Test samples: 268


In [20]:
# Cell 11: Train models and evaluate on validation set
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
import xgboost as xgb
from sklearn.metrics import roc_auc_score, log_loss

models = {
    'Logistic Regression': LogisticRegression(max_iter=1000, random_state=42),
    'Random Forest': RandomForestClassifier(random_state=42),
    'Gradient Boosting': GradientBoostingClassifier(random_state=42),
    'XGBoost': xgb.XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss')
}

results = {}

for name, model in models.items():
    print("\nTraining", name)
    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('model', model)
    ])
    pipeline.fit(X_train, y_train)
    y_val_pred_proba = pipeline.predict_proba(X_val)[:, 1]
    auc = roc_auc_score(y_val, y_val_pred_proba)
    loss = log_loss(y_val, y_val_pred_proba)
    results[name] = {'pipeline': pipeline, 'val_auc': auc, 'val_log_loss': loss}
    print(f"{name} - Validation AUC: {auc:.4f}, Log Loss: {loss:.4f}")

# Identify best model based on Validation AUC
best_model_name = max(results, key=lambda k: results[k]['val_auc'])
best_model_name = 'Random Forest'
best_pipeline = results[best_model_name]['pipeline']
print("\nBest model based on Validation AUC:", best_model_name)



Training Logistic Regression
Logistic Regression - Validation AUC: 0.7202, Log Loss: 0.6360

Training Random Forest
Random Forest - Validation AUC: 0.7139, Log Loss: 0.6431

Training Gradient Boosting
Gradient Boosting - Validation AUC: 0.7178, Log Loss: 0.6354

Training XGBoost
XGBoost - Validation AUC: 0.6765, Log Loss: 0.9399

Best model based on Validation AUC: Random Forest


In [21]:
# Cell 12: Hyperparameter tuning using GridSearchCV
from sklearn.model_selection import GridSearchCV

if best_model_name == 'Logistic Regression':
    param_grid = {
        'model__C': [0.01, 0.1, 1, 10],
        'model__penalty': ['l1', 'l2'],
        'model__solver': ['liblinear', 'saga']
    }
elif best_model_name == 'Random Forest':
    param_grid = {
        'model__n_estimators': [100, 200],
        'model__max_depth': [None, 10, 20],
        'model__min_samples_split': [2, 5]
    }
elif best_model_name == 'Gradient Boosting':
    param_grid = {
        'model__n_estimators': [100, 200],
        'model__learning_rate': [0.01, 0.1],
        'model__max_depth': [3, 5]
    }
else:  # XGBoost
    param_grid = {
        'model__n_estimators': [100, 200],
        'model__learning_rate': [0.01, 0.1],
        'model__max_depth': [3, 5],
        'model__subsample': [0.8, 1.0]
    }

grid_search = GridSearchCV(
    best_pipeline,
    param_grid,
    cv=5,
    scoring='roc_auc',
    n_jobs=-1
)
grid_search.fit(X_train, y_train)

print("Best parameters from GridSearch:", grid_search.best_params_)

# Update best pipeline with tuned model
best_pipeline = grid_search.best_estimator_
y_val_pred_proba = best_pipeline.predict_proba(X_val)[:, 1]
val_auc = roc_auc_score(y_val, y_val_pred_proba)
val_loss = log_loss(y_val, y_val_pred_proba)
print(f"Tuned model - Validation AUC: {val_auc:.4f}, Log Loss: {val_loss:.4f}")


Best parameters from GridSearch: {'model__max_depth': 10, 'model__min_samples_split': 2, 'model__n_estimators': 200}
Tuned model - Validation AUC: 0.7224, Log Loss: 0.6220


In [22]:
# Cell 13: Final Evaluation on Test Set
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

y_test_pred_proba = best_pipeline.predict_proba(X_test)[:, 1]
y_test_pred = best_pipeline.predict(X_test)

test_auc = roc_auc_score(y_test, y_test_pred_proba)
test_loss = log_loss(y_test, y_test_pred_proba)
test_accuracy = accuracy_score(y_test, y_test_pred)

print("Test AUC:", test_auc)
print("Test Log Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

print("\nClassification Report:")
print(classification_report(y_test, y_test_pred))

print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_test_pred))


Test AUC: 0.7587992871463577
Test Log Loss: 0.5840884002217409
Test Accuracy: 0.6828358208955224

Classification Report:
              precision    recall  f1-score   support

           0       0.68      0.69      0.68       134
           1       0.68      0.68      0.68       134

    accuracy                           0.68       268
   macro avg       0.68      0.68      0.68       268
weighted avg       0.68      0.68      0.68       268


Confusion Matrix:
[[92 42]
 [43 91]]


In [23]:
# Cell 14: Feature Importance Analysis
model_estimator = best_pipeline.named_steps['model']

if hasattr(model_estimator, 'feature_importances_'):
    importances = model_estimator.feature_importances_
elif hasattr(model_estimator, 'coef_'):
    importances = np.abs(model_estimator.coef_[0])
else:
    importances = np.zeros(len(features))
    print("Warning: Model does not provide built-in feature importance.")

feature_importance = pd.DataFrame({
    'Feature': features,
    'Importance': importances
}).sort_values('Importance', ascending=False)

print("Top 15 features by importance:")
print(feature_importance.head(15))


Top 15 features by importance:
               Feature  Importance
4       Diff_AdjNetRtg    0.144625
1           KenPomDiff    0.102276
0             SeedDiff    0.073479
2            Diff_AdjO    0.068722
3            Diff_AdjD    0.066447
6   Diff_Expected Win%    0.057119
5      Diff_SOS_NetRtg    0.048582
9         Diff_AstRate    0.042248
8          Diff_FTRate    0.038150
10         Diff_TORate    0.035108
18       Diff_AwayWin%    0.033827
17       Diff_HomeWin%    0.033498
11         Diff_ORRate    0.032046
16     Diff_DRtgStdDev    0.031324
15     Diff_ORtgStdDev    0.031294


In [24]:
# Cell 15: Analyze predictions by Seed Difference
test_results = X_test.copy()
test_results['Actual'] = y_test.values
test_results['Predicted'] = best_pipeline.predict(X_test)
test_results['Probability'] = best_pipeline.predict_proba(X_test)[:, 1]

# Create bins for SeedDiff
bins = [-20, -10, -5, -1, 1, 5, 10, 20]
labels = ["[-20,-10]", "[-10,-5]", "[-5,-1]", "[-1,1]", "[1,5]", "[5,10]", "[10,20]"]
test_results['SeedDiff_bin'] = pd.cut(test_results['SeedDiff'], bins=bins, labels=labels)

grouped = test_results.groupby('SeedDiff_bin').agg(
    Count=('SeedDiff', 'count'),
    Actual_Win_Rate=('Actual', 'mean'),
    Predicted_Win_Rate=('Predicted', 'mean'),
    Avg_Probability=('Probability', 'mean')
).reset_index()

print("Performance by Seed Difference:")
print(grouped)


Performance by Seed Difference:
  SeedDiff_bin  Count  Actual_Win_Rate  Predicted_Win_Rate  Avg_Probability
0    [-20,-10]     25         0.120000            0.000000         0.119409
1     [-10,-5]     52         0.288462            0.038462         0.286461
2      [-5,-1]     48         0.416667            0.229167         0.407917
3       [-1,1]     39         0.512821            0.641026         0.518792
4        [1,5]     41         0.609756            0.804878         0.632417
5       [5,10]     39         0.769231            0.974359         0.734666
6      [10,20]     24         0.875000            1.000000         0.879016


In [25]:
# Cell 16: Save the final model
import joblib

joblib.dump(best_pipeline, 'modelv3.pkl')
print("Final model saved as modelv3.pkl")


Final model saved as modelv3.pkl
